Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator` and set it to `GPU`.

In [32]:
!nvidia-smi


Sun Jun 30 00:41:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8             10W /  105W |    4866MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Install Detectron2 and dependencies

In [33]:
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\daanish mittal\appdata\local\temp\pip-req-build-w6nilpdy
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\Daanish Mittal\AppData\Local\Temp\pip-req-build-w6nilpdy\setup.py", line 10, in <module>
        import torch
    ModuleNotFoundError: No module named 'torch'
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\Daanish Mittal\AppData\Local\Temp\pip-req-build-w6nilpdy\
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Now is a good time to confirm that we have the right versions of the libraries at our disposal.

In [34]:
!pip install -U "iopath<0.1.10,>=0.1.7"
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
import pkg_resources
print("detectron2:", pkg_resources.get_distribution("detectron2").version)

Requirement already up-to-date: iopath<0.1.10,>=0.1.7 in c:\users\daanish mittal\appdata\local\programs\python\python36\lib\site-packages (0.1.9)


You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Fri_Feb__8_19:08:26_Pacific_Standard_Time_2019
Cuda compilation tools, release 10.1, V10.1.105
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [35]:
# COMMON LIBRARIES
import os
import cv2

from datetime import datetime
import cv2

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

## Run a Pre-trained Detectron2 Model

In [36]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
# image = cv2.imread("./input.jpg")
# cv2.imshow('Display Window', image)  # 'Display Window' is the name of the window
# cv2.waitKey(0)  # Waits for a key press to close the window
# cv2.destroyAllWindows()  # Closes the window

In [37]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade --force-reinstall

Looking in indexes: https://download.pytorch.org/whl/cu117


  Could not find a version that satisfies the requirement torch (from versions: )
No matching distribution found for torch
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## COCO Format Dataset

In [38]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RovqaIFPpcekpUVWjRke")
project = rf.workspace("tank-5yib6").project("tank_edges")
version = project.version(1)
dataset = version.download("coco-segmentation")


  Could not find a version that satisfies the requirement supervision (from roboflow) (from versions: )
No matching distribution found for supervision (from roboflow)
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/99/f2/8349482b1cb7d408cd8673efaa1b51e7bd657f95b679de27f89e333f1388/roboflow-1.1.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e4/73/b46073efb9ab0e07dae64d5b0229825eb19cc393d9f5ba8ec9d693e4e5f3/opencv-python-headless-4.8.0.74.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Using cached https://files.pythonhosted.org/packages/71/4c/3db2b8021bd6f2f0ceb0e088d6b2d49147671f25832fb17970e9b583d742/certifi-2022.12.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/51/d4db610ef29373b879047326cbf6fa98b6c1969d6f6dc423279de2b1be2c/requests_toolbelt-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/61/08076519c80041bc0ffa1a8af0cbd3bf3e2b62af10435

### Register

When you use Detectron2, before you actually train the model you need to [register it](https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html#register-a-coco-format-dataset).

In [39]:
DATA_SET_NAME = dataset.name.replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

In [40]:
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME,
    metadata={},
    json_file=TEST_DATA_SET_ANN_FILE_PATH,
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

We can now confirm that our custom dataset was correctly registered using [MetadataCatalog](https://detectron2.readthedocs.io/en/latest/modules/data.html#detectron2.data.MetadataCatalog).

In [41]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

['tank_edges-train', 'tank_edges-test', 'tank_edges-valid']

### Visualize

Let's take a look at single entry from out train dataset.

In [46]:
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, ::-1],
    metadata=metadata,
    scale=0.5,
    instance_mode=ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
cv2.imshow("visualize",out.get_image()[:, :, ::-1])
cv2.waitKey(0)  # Waits for a key press to close the window
cv2.destroyAllWindows()  # Closes the window

WARNING [06/30 01:01:22 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/30 01:01:22 d2.data.datasets.coco]: Loaded 180 images in COCO format from d:\Tank_proj\tank_edge\tank_edges-1\train\_annotations.coco.json


## Train Model Using Custom COCO Format Dataset

### Configuration

In [52]:
# HYPERPARAMETERS
ARCHITECTURE = "mask_rcnn_R_101_FPN_3x"
CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
MAX_ITER = 2000
EVAL_PERIOD = 300
BASE_LR = 0.001
NUM_CLASSES = 3

# OUTPUT DIRa
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [ ]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
# cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
# cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
# cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.INPUT.MASK_FORMAT='bitmask'
# cfg.SOLVER.BASE_LR = BASE_LR
# cfg.SOLVER.MAX_ITER = MAX_ITER
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
# cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

In [53]:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # Default batch size per image

# Adjusting batch size and image size for GPU with 6GB VRAM
cfg.SOLVER.IMS_PER_BATCH = 2  # Overall batch size
cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)  # Adjusted to fit memory constraints
cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MAX_SIZE_TEST = 1333

cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.DATALOADER.NUM_WORKERS = 3
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.INPUT.MASK_FORMAT = 'bitmask'
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

# Save config for future reference
with open(os.path.join(OUTPUT_DIR_PATH, "config.yaml"), "w") as f:
    f.write(cfg.dump())

### Training

In [54]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[06/30 01:03:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[06/30 01:03:55 d2.engine.train_loop]: Starting training from iteration 0
[06/30 01:04:48 d2.utils.events]:  eta: 1:22:07  iter: 19  total_loss: 2.296  loss_cls: 1.361  loss_box_reg: 0.2658  loss_mask: 0.6846  loss_rpn_cls: 0.00204  loss_rpn_loc: 0.002668    time: 2.5426  last_time: 2.9995  data_time: 0.1222  last_data_time: 0.0017   lr: 1.9981e-05  max_mem: 5719M
[06/30 01:05:51 d2.utils.events]:  eta: 1:34:17  iter: 39  total_loss: 1.618  loss_cls: 0.8273  loss_box_reg: 0.2199  loss_mask: 0.5762  loss_rpn_cls: 0.007008  loss_rpn_loc: 0.002509    time: 2.8544  last_time: 3.1060  data_time: 0.0017  last_data_time: 0.0015   lr: 3.9961e-05  max_mem: 5719M
[06/30 01:06:47 d2.utils.events]:  eta: 1:32:44  iter: 59  total_loss: 1.023  loss_cls: 0.3645  loss_box_reg: 0.2358  loss_mask: 0.4276  loss_rpn_cls: 0.00607  loss_rpn_loc: 0.002897    time: 2.8278  last_time: 2.7016  data_time: 0.0018  last_data_time: 0.0019   lr: 5.9941e-05  max_mem: 5720M
[06/30 01:07:38 d2.utils.events]:  eta: 1:28

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

### Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
dataset_valid = DatasetCatalog.get(VALID_DATA_SET_NAME)

for d in dataset_valid:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata=metadata,
        scale=0.8,
        instance_mode=ColorMode.IMAGE_BW
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("validation",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)  # Waits for a key press to close the window
    cv2.destroyAllWindows()  # Closes the window

In [4]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97
      # Set a custom testing threshold
    return DefaultPredictor(cfg)

def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Visualize the results
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image
    result_image = out.get_image()[:, :, ::-1]
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        # Display the frame
        result_frame = out.get_image()[:, :, ::-1]
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-25-16-10-19/config.yaml"
weights_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-25-16-10-19/model_final.pth"
# config_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-28-12-40-23/config.yaml"
# weights_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-28-12-40-23/model_final.pth"

input_folder = "D:/path_detect/test"

predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)


In [3]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97  # Set a custom testing threshold
    return DefaultPredictor(cfg)

def draw_quadrilaterals(image, outputs, deformation_threshold=0.05):
    masks = outputs["instances"].pred_masks.to("cpu").numpy()
    for mask in masks:
        # Find contours
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            continue
        
        for contour in contours:
            # Approximate the contour to a quadrilateral
            epsilon = deformation_threshold * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Ensure it's a quadrilateral (4 sides)
            if len(approx) == 4:
                # Draw the quadrilateral
                cv2.polylines(image, [approx], True, (0, 255, 0), 2)
                
    return image


def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Draw quadrilaterals and masks on the image
    result_image = draw_quadrilaterals_and_masks(image, outputs)
    
    # Display the image
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        result_frame = draw_quadrilaterals_and_masks(frame, outputs)
        
        # Display the frame
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-25-16-10-19/config.yaml"
weights_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-25-16-10-19/model_final.pth"

input_folder = "D:/path_detect/test"

predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)


NameError: name 'draw_quadrilaterals_and_masks' is not defined

In [2]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.95  # Set a custom testing threshold
    return DefaultPredictor(cfg)

def draw_quadrilaterals(image, outputs):
    masks = outputs["instances"].pred_masks.to("cpu").numpy()
    for mask in masks:
        # Find contours
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            continue
        
        for contour in contours:
            # Approximate the contour to a quadrilateral
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Calculate the area of the mask and the quadrilateral
            mask_area = cv2.contourArea(contour)
            quad_area = cv2.contourArea(approx)
            
            # Check if mask_area is zero (handle edge case)
            if mask_area <= 0:
                continue
            
            # Calculate the percentage of the mask area outside the quadrilateral
            outside_area = mask_area - quad_area
            outside_percentage = outside_area / mask_area if mask_area > 0 else 0
            
            # Determine the color of the quadrilateral
            color = (0, 255, 0) if outside_percentage <= 0.12 else (0, 0, 255)
            
            if len(approx) >= 4:
                cv2.polylines(image, [approx], True, color, 2)
                
    return image


def process_image(predictor, image_path, output_folder):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Draw quadrilaterals on the image
    result_image = draw_quadrilaterals(image, outputs)
    
    # Save the result image
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, result_image)
    print(f'Saved result for {image_path} to {output_path}')

def process_video(predictor, video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    output_path = os.path.join(output_folder, os.path.basename(video_path))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        result_frame = draw_quadrilaterals(frame, outputs)
        
        # Write the frame to the output video
        out.write(result_frame)
    
    cap.release()
    out.release()
    print(f'Saved result for {video_path} to {output_path}')

def process_folder(predictor, folder_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, output_folder)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path, output_folder)

# Example usage
config_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-25-16-10-19/config.yaml"
weights_file = "D:/path_detect/tank/mask_rcnn_R_101_FPN_3x/2024-06-25-16-10-19/model_final.pth"

input_folder = "D:/path_detect/test"
output_folder = "D:/path_detect/output"

predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder, output_folder)


c:\Users\Daanish Mittal\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Saved result for D:/path_detect/test\IMG1551952615468_jpg.rf.18ba64b8924724efc330d01981c585bd.jpg to D:/path_detect/output\IMG1551952615468_jpg.rf.18ba64b8924724efc330d01981c585bd.jpg
Saved result for D:/path_detect/test\IMG1554039051734_jpg.rf.347e3b8c0f42e174cf14976b3a98d14c.jpg to D:/path_detect/output\IMG1554039051734_jpg.rf.347e3b8c0f42e174cf14976b3a98d14c.jpg
Saved result for D:/path_detect/test\WhatsApp Image 2024-06-25 at 23.15.50_4d2547a6.jpg to D:/path_detect/output\WhatsApp Image 2024-06-25 at 23.15.50_4d2547a6.jpg
Saved result for D:/path_detect/test\WhatsApp Image 2024-06-25 at 23.43.27_aa0561da.jpg to D:/path_detect/output\WhatsApp Image 2024-06-25 at 23.43.27_aa0561da.jpg
Saved result for D:/path_detect/test\WhatsApp Image 2024-06-25 at 23.49.54_963bf197.jpg to D:/path_detect/output\WhatsApp Image 2024-06-25 at 23.49.54_963bf197.jpg
Saved result for D:/path_detect/test\WhatsApp Image 2024-06-28 at 16.00.55_072c0a4e.jpg to D:/path_detect/output\WhatsApp Image 2024-06-28 a

In [ ]:
import cv2
import numpy as np

def detect_ramp_edges(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Detect edges using Canny
    edges = cv2.Canny(blurred, 300, 250, apertureSize=3)

    # Create a mask to remove boundary edges
    height, width = edges.shape
    mask = np.ones_like(edges)
    border = 10  # Adjust this value to change the border size
    mask[:border, :] = 0
    mask[-border:, :] = 0
    mask[:, :border] = 0
    mask[:, -border:] = 0

    # Apply the mask to remove boundary edges
    edges = cv2.bitwise_and(edges, edges, mask=mask)


    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1.1, np.pi / 180,200)


    return lines

def draw_lines(image, lines):
    # Draw the lines on the image
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return image

def process_video(video_source=0):
    # Open the video source (0 for webcam, or a filename)
    cap = cv2.VideoCapture(video_source)

    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect ramp edges
        lines = detect_ramp_edges(frame)

        # Draw detected lines on the frame
        frame_with_lines = draw_lines(frame, lines)

        # Display the result
        cv2.imshow('Detected Ramp Edges', frame_with_lines)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    process_video('D:/path_detect/video1.mp4')

In [1]:
import cv2
import numpy as np

def detect_ramp_edges(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Detect edges using Canny
    edges = cv2.Canny(blurred, 100, 200, apertureSize=3)

    # Create a mask to remove boundary edges
    height, width = edges.shape
    mask = np.ones_like(edges)
    border = 10  # Adjust this value to change the border size
    mask[:border, :] = 0
    mask[-border:, :] = 0
    mask[:, :border] = 0
    mask[:, -border:] = 0

    # Apply the mask to remove boundary edges
    edges = cv2.bitwise_and(edges, edges, mask=mask)

    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1.1, np.pi / 180, 200)

    return lines

def draw_lines(image, lines):
    # Draw the lines on the image
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return image

def process_video(video_source=0):
    # Open the video source (0 for webcam, or a filename)
    cap = cv2.VideoCapture(video_source)

    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect ramp edges
        lines = detect_ramp_edges(frame)

        # Draw detected lines on the frame
        frame_with_lines = draw_lines(frame, lines)

        # Display the result
        cv2.imshow('Detected Ramp Edges', frame_with_lines)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    process_video()  # Default is the webcam, pass a video file path if needed
